# [11. Python Advanced Modules II.](https://docs.python.org/3/tutorial/stdlib2.html)
- Support professional programming needs
- These modules rarely occur in small scripts

## 11.1. Output Formatting
`reprlib` - [creates abrecations in Alphabetical order](https://docs.python.org/3/library/reprlib.html#module-reprlib)

In [303]:
import reprlib
reprlib.repr(set('supercalifragilisticexpialidocious'))

"{'a', 'c', 'd', 'e', 'f', 'g', ...}"

`pprint` module
- sophisticated control over printing both 
    - built-in and 
    - user defined objects 
- in a way that is readable by the interpreter. 
- When the result is longer than one line, the “pretty printer” adds line breaks and indentation to more clearly reveal data structure:

In [304]:
import pprint
t = [[[['black', 'cyan'], 'white', ['green', 'red']], [['magenta',
    'yellow'], 'blue']]]

pprint.pprint(t, width=30)

[[[['black', 'cyan'],
   'white',
   ['green', 'red']],
  [['magenta', 'yellow'],
   'blue']]]


`textwrap` - [module](https://docs.python.org/3/library/textwrap.html#module-textwrap) 
- formats paragraphs of text to 
- fit a given screen width

In [305]:
import textwrap
doc = """The wrap() method is just like fill() except that it returns
a list of strings instead of one big string with newlines to separate
the wrapped lines."""

print(textwrap.fill(doc, width=40))

The wrap() method is just like fill()
except that it returns a list of strings
instead of one big string with newlines
to separate the wrapped lines.


`locale` - [module](https://docs.python.org/3/library/locale.html#module-locale) 
- accesses a database of culture specific data formats
- ex.: direct way of formatting numbers with group separators:

In [306]:
import locale
locale.setlocale(locale.LC_ALL, '')

'Hungarian_Hungary.1250'

In [307]:
conv = locale.localeconv()          # get a mapping of conventions
x = 1234567.8
locale.format("%d", x, grouping=True)

X:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: This method will be removed in a future version of Python. Use 'locale.format_string()' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


'1\xa0234\xa0567'

In [308]:
locale.format_string("%s%.*f", (conv['currency_symbol'],
                     conv['frac_digits'], x), grouping=True)

'Ft1\xa0234\xa0567,80'

## 11.2. Template

### `$` - placeholder: put whatever you want there

In [309]:
from string import Template
t = Template('${village}folk send $$10 to $cause.')
t.substitute(village='Nottingham', cause='the ditch fund')

'Nottinghamfolk send $10 to the ditch fund.'

In [310]:
t = Template('Return the $item to $owner.')
d = dict(item='unladen swallow')

In [311]:
t.safe_substitute(d)

'Return the unladen swallow to $owner.'

### Renaming photo batch

In [312]:
import time, os.path
photofiles = ['img_1074.jpg', 'img_1076.jpg', 'img_1077.jpg']
class BatchRename(Template):
    delimiter = '%'
fmt = input('Enter rename style (%d-date %n-seqnum %f-format):  ')

Enter rename style (%d-date %n-seqnum %f-format):   1994 Hello f


In [313]:
t = BatchRename(fmt)
date = time.strftime('%d%b%y')
for i, filename in enumerate(photofiles):
    base, ext = os.path.splitext(filename)
    newname = t.substitute(d=date, n=i, f=ext)
    print('{0} --> {1}'.format(filename, newname))

img_1074.jpg --> 1994 Hello f
img_1076.jpg --> 1994 Hello f
img_1077.jpg --> 1994 Hello f


## 11.3. Binary Data
[pack()](https://docs.python.org/3/library/struct.html#struct.pack)

[unpack()](https://docs.python.org/3/library/struct.html#struct.unpack)

import struct

with open('myfile.zip', 'rb') as f:
    data = f.read()

start = 0
for i in range(3):                      # show the first 3 file headers
    start += 14
    fields = struct.unpack('<IIIHH', data[start:start+16])
    crc32, comp_size, uncomp_size, filenamesize, extra_size = fields

    start += 16
    filename = data[start:start+filenamesize]
    start += filenamesize
    extra = data[start:start+extra_size]
    print(filename, hex(crc32), comp_size, uncomp_size)

    start += extra_size + comp_size     # skip to the next header

## 11.4. Multi-threading | | | |
decoupling NOT sequentially tasks

`queue` - [feeds the therad](https://docs.python.org/3/library/queue.html#module-queue)

`threading` - [run tasks in background while the main program continues to run:](https://docs.python.org/3/library/threading.html#module-threading)

In [314]:
import threading, zipfile

class AsyncZip(threading.Thread):
    def __init__(self, infile, outfile):
        threading.Thread.__init__(self)
        self.infile = infile
        self.outfile = outfile

    def run(self):
        f = zipfile.ZipFile(self.outfile, 'w', zipfile.ZIP_DEFLATED)
        f.write(self.infile)
        f.close()
        print('Finished background zip of:', self.infile)

background = AsyncZip('mydata.txt', 'myarchive.zip')
background.start()
print('The main program continues to run in foreground.')

background.join()    # Wait for the background task to finish
print('Main program waited until background was done.')

The main program continues to run in foreground.
Finished background zip of: mydata.txt
Main program waited until background was done.


## 11.5. Logging
flexible logging system

`sys.stderr` log messages sent

Messages: DEBUG, INFO, WARNING, ERROR, and CRITICAL

In [315]:
import logging
logging.debug('Debugging information')
logging.info('Informational message')
logging.warning('Warning:config file %s not found', 'server.conf')
logging.error('Error occurred')
logging.critical('Critical error -- shutting down')

ERROR:root:Error occurred
CRITICAL:root:Critical error -- shutting down


## 11.6. Weak References
`weakref` [module](https://docs.python.org/3/library/weakref.html#module-weakref) 
- provides tools for tracking objects 
- without creating a reference
- automatic memory management (Python)
- [garbage collection](https://docs.python.org/3/glossary.html#term-garbage-collection)


In [316]:
import weakref, gc
class A:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return str(self.value)

a = A(10)                   # create a reference
d = weakref.WeakValueDictionary()
d['primary'] = a            # does not create a reference
d['primary']                # fetch the object if it is still alive

10

In [317]:
del a                       # remove the one reference
gc.collect()                # run garbage collection right away

66

In [318]:
d['primary']                # entry was automatically removed

10

## 11.7. `List` tools
`array` - [moule](https://docs.python.org/3/library/array.html#module-array)
- array() object that is like a list that stores only homogeneous data 
- more compactly
- [array class sub objects](https://docs.python.org/3/library/array.html#array.array)
    - array.typecodes
    - array.typecode
    - array.itemsize
    - array.append(x)
    - array.buffer_info()
    - array.byteswap()
    - array.count(x)


In [319]:
from array import array
a = array('H', [4000, 10, 700, 22222]) #stored as two byte unsigned binary numbers
sum(a)

26932

In [320]:
a[1:3]

array('H', [10, 700])

In [321]:
from collections import deque
d = deque(["task1", "task2", "task3"])
d.append("task4")
print("Handling", d.popleft())

Handling task1


`bisect` - [module](https://docs.python.org/3/library/bisect.html#module-bisect)
- manipulate sorted lists

In [322]:
import bisect
scores = [(100, 'perl'), (200, 'tcl'), (400, 'lua'), (500, 'python')]
bisect.insort(scores, (300, 'ruby'))
scores

[(100, 'perl'), (200, 'tcl'), (300, 'ruby'), (400, 'lua'), (500, 'python')]

`heapq` - [module](https://docs.python.org/3/library/heapq.html#module-heapq)

In [323]:
from heapq import heapify, heappop, heappush
data = [1, 3, 5, 7, 9, 2, 4, 6, 8, 0]
heapify(data)                      # rearrange the list into heap order
heappush(data, -5)                 # add a new entry
[heappop(data) for i in range(3)]  # fetch the three smallest entries

[-5, 0, 1]

## 11.8. Decimal Arithmetic

`decimal` - [module](https://docs.python.org/3/library/decimal.html#module-decimal)

`Decimal` - [datatype](https://docs.python.org/3/library/decimal.html#decimal.Decimal)

### Used at:
- financial applications and other uses which require exact decimal representation,
- control over precision,
- control over rounding to meet legal or regulatory requirements,
- tracking of significant decimal places, or
- applications where the user expects the results to match calculations done by hand.

calculating a 5% tax on a 70 cent phone charge gives different results in decimal floating point and binary floating point. The difference becomes significant if the results are rounded to the nearest cent:

In [324]:
from decimal import *
round(Decimal('0.70') * Decimal('1.05'), 2)

Decimal('0.74')

In [325]:
round(.70 * 1.05, 2)

0.73

In [326]:
Decimal('1.00') % Decimal('.10')

Decimal('0.00')

In [327]:
1.00 % 0.10

0.09999999999999995

In [328]:
sum([Decimal('0.1')]*10) == Decimal('1.0')

True

In [329]:
sum([0.1]*10) == 1.0

False

In [330]:
getcontext().prec = 36
Decimal(1) / Decimal(7)

Decimal('0.142857142857142857142857142857142857')

## 11.9. Floating Point Arithmetic
`float` - [module](https://docs.python.org/3/library/functions.html#float)